In [1]:
import numpy as np

import matplotlib.pylab as pl
import matplotlib.pyplot as plt
from math import log2, ceil
import seaborn as sns
import pandas as pd

from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
cmap_light = ListedColormap(['#FFBBBB', '#BBFFBB', '#BBBBFF'])
cmap_bold = ListedColormap(['#CC0000', '#00AA00', '#0000CC'])

In [2]:
import functions.cc18_with_cpd_functions as fn
from proglearn.sims import generate_gaussian_parity
from proglearn.progressive_learner import ProgressiveLearner
from proglearn.deciders import SimpleArgmaxAverage
from proglearn.transformers import (
    TreeClassificationTransformer,
    NeuralClassificationTransformer,
)
from proglearn.voters import TreeClassificationVoter, KNNClassificationVoter
from proglearn.sims import generate_gaussian_parity

ModuleNotFoundError: No module named 'functions'

## Import

In [ ]:
df = pd.read_csv('php0iVrYT.csv');

data = df.values[:, :4].astype(int);
target = df.values[:, 4].astype(int);
d_name = np.array(df.columns[:4]);
t_name = np.array(df.columns[4]);

## Visualization

In [ ]:
#plot two features
i, j = 0, 1;

plt.figure(figsize=(10,5));
plt.scatter(data[:,i], data[:,j], edgecolor='none', c=target);

plt.xlabel(d_name[i]);
plt.ylabel(d_name[j]);
plt.title(d_name[i] + ' vs ' + d_name[j]);

In [ ]:
#explained variance
from sklearn.decomposition import PCA

X = data.copy();
pca = PCA(n_components = 3, whiten = False);
X = pca.fit_transform(X);

E, L = pca.components_.T, pca.explained_variance_
cl = np.cumsum(L);

plt.subplot(211);
plt.plot(L,'o-');
plt.ylabel('Eigenvalues');

plt.subplot(212);
plt.plot(cl/cl[-1],'o-r');
plt.ylabel('Total Variance');

# Functions

In [ ]:
def rotate_dist(X, theta=0, dim=[0,1]):
    #dimensions to rotate
    Z = X[:, dim]
    
    #rotation
    c, s = np.cos(theta), np.sin(theta)
    R = np.array([[c, -s], [s, c]])
    Z = Z @ R
    
    #recombine
    X[:, dim] = Z
    
    return X

In [ ]:
def shear_dist(X, s=0, dim=[0,1]):
    #dimensions to shear
    Z = X[:, dim]
    
    #shear
    R = np.array([[1, s], [s, 1]])
    Z = Z @ R
    
    #recombine
    X[:, dim] = Z
    
    return X

In [ ]:
def split_task(X, y):
    D = np.hstack((X, y))
    np.random.shuffle(D)
    
    n,_ = np.shape(D)
    m = int(n/2)
    D1 = D[:m, :]
    D2 = D[m:, :]
    
    X1 = D1[:, :-1]
    X2 = D2[:, :-1]
    
    y1 = D1[:, -1]
    y2 = D2[:, -1]
    
    return X1, y1, X2, y2

In [ ]:
def sample_cc18(frac=0.6, angle_params=0, shear_params=0, dimension=[0,1]):
    df = pd.read_csv('php0iVrYT.csv')

    data = df.values.astype(int)
    np.random.shuffle(data)
    
    n,d = np.shape(data)
    m = int(frac * n)
    
    X = data[:m, :d-1]
    y = data[:m, d-1]
    
    pca = PCA(n_components = 3, whiten = False)
    X = pca.fit_transform(X)
    
    X = rotate_dist(X, angle_params, dimension)
    X = shear_dist(X, shear_params, dimension)
    
    return X, y

In [ ]:
X1, y1 = sample_cc18(frac=0.6)
X2, y2 = sample_cc18(frac=0.6, angle_params=np.pi/4, shear_params=0)
X3, y3 = sample_cc18(frac=0.6, angle_params=0, shear_params=1/4)

plt.figure(figsize=(8, 20))
plt.subplot(311)
plt.scatter(X1[:,0], X1[:,1], c=y1, s=50)
plt.title('Task 1');

plt.subplot(312)
plt.scatter(X2[:,0], X2[:,1], c=y2, s=50)
plt.title('Task 2 (rotation)');

plt.subplot(313)
plt.scatter(X3[:,0], X3[:,1], c=y3, s=50)
plt.title('Task 3 (shear)');

In [ ]:
angle_sweep = range(0,181,10)
task1_frac = 0.6
task2_frac = 0.6
mc_rep = 250

In [ ]:
mean_te1 = fn.bte_v_angle(angle_sweep, task1_sample, task2_sample, mc_rep, register_cpd=False, register_otp=False)

In [ ]:
mean_te2 = fn.bte_v_angle(angle_sweep, task1_sample, task2_sample, mc_rep, register_cpd=True)

In [ ]:
mean_te3 = fn.bte_v_angle(angle_sweep, task1_sample, task2_sample, mc_rep, register_otp=True)

In [ ]:
mean_te4 = fn.bte_v_angle(angle_sweep, task1_sample, task2_sample, mc_rep, register_icp=True)

In [ ]:
fontsize=24
fig, ax = plt.subplots(1, 1, figsize=(16, 16))
task = ["No adaptation", "ICP", "CPD", "OT-exact", "OT-IT"]
ax.plot(angle_sweep, mean_te1, linewidth=3, label=task[0])
ax.plot(angle_sweep, mean_te4, linewidth=3, label=task[1])
ax.plot(angle_sweep, mean_te2, linewidth=3, label=task[2])
ax.plot(angle_sweep, mean_te3, linewidth=3, label=task[4])
ax.set_xticks(range(0, 181, 10))
ax.hlines(1, 0, 180, colors="gray", linestyles="dashed", linewidth=1.5)
ax.set_xlabel("Angle of Shear (Degrees)",fontsize=fontsize)
ax.set_ylabel("Backward Transfer Efficiency (CC18 w/Shear)",fontsize=fontsize)
ax.legend(loc="lower left", fontsize=20, frameon=False)

right_side = ax.spines["right"]
right_side.set_visible(False)
top_side = ax.spines["top"]
top_side.set_visible(False)